In [1]:
def create_triples(df):
    """
    df: DataFrame with at least 'SMILES' and 'use_case' columns
    Returns: DataFrame with weighted relationships
    """
    # Count occurrences of each SMILES-use_case pair
    triples = df.groupby(['SMILES', 'Harmonized Functional Use']).size().reset_index()
    triples.columns = ['SMILES', 'Harmonized Functional Use', 'count']
    
    # Save to CSV
    triples.to_csv('CPDatMolecule_use_triples.csv', index=False)
    return triples

In [2]:
import pandas as pd 

In [3]:
df = pd.read_csv('../../Data/Curated/UseCaseDataModeling.csv')

/tmp/ipykernel_1108315/864759677.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/matt/Proj/QSURv3/Data/Curated/UseCaseDataModeling.csv')


In [4]:
df.columns

Index(['Data Source', 'Data Type', 'Data Document ID', 'Data Document Title',
       'Document Date', 'DTXSID', 'Curated Chemical Name', 'CAS',
       'Provisional DTXSID', 'Harmonized Functional Use',
       ...
       'Bit_2040', 'Bit_2041', 'Bit_2042', 'Bit_2043', 'Bit_2044', 'Bit_2045',
       'Bit_2046', 'Bit_2047', 'SMILES', 'Harmonized Functional Use Encoded'],
      dtype='object', length=2060)

In [5]:
trip = create_triples(df)

In [6]:
trip

,SMILES,Harmonized Functional Use,count
0,B#N,Absorbent,2
1,B#N,Anti-adhesive/cohesive,1
2,B#N,Conductive agent,4
3,B#N,Crystal growth modifiers (nucleating agents),1
4,B#N,Film former,1
...,...,...,...
20478,[Zn],Plating agent,4
20479,[Zn],Surface modifier,2
20480,[Zn].[Ba],Heat stabilizer,4
20481,[Zn].[Ba],UV stabilizer,3


In [10]:
trip.to_csv('triples.csv')

In [7]:
from neo4j import GraphDatabase

def create_graph_from_triples(csv_path, uri="bolt://localhost:7687", user="neo4j", password="matt1234"):
    # Connect to Neo4j
    driver = GraphDatabase.driver(uri, auth=(user, password))
    
    # Read CSV
    triples_df = pd.read_csv(csv_path)
    


    def create_relationship(tx, subject, object, count):
        # Cypher query to create/merge nodes and relationships
        query = """
        MERGE (s:Node {name: $subject})
        MERGE (o:Node {name: $object})
        MERGE (s)-[r:RELATES_TO]->(o)
        SET r.count = $count
        """
        tx.run(query, subject=subject, object=object, count=count)
    
    # Create relationships for each triple
    with driver.session() as session:
        
        for _, row in triples_df.iterrows():
            session.execute_write(
                create_relationship, 
                row['SMILES'], 
                row['reported_function'], 
                row['count']
            )
    
    driver.close()

# Usage:
# create_graph_from_triples('your_triples.csv')

In [13]:
create_graph_from_triples('triples.csv')

Transaction failed and will be retried in 1.0034514412325506s (Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))
Transaction failed and will be retried in 2.3518087317869005s (Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))
Transaction failed and will be retried in 4.382152158284542s (Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))
Transaction failed and will be retried in 9.384328805514608s (Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))
Transaction failed and will be retried in 18.81476477122756s (Couldn't con

KeyboardInterrupt: 